<a href="https://colab.research.google.com/github/22222bh/SketchTogether/blob/user%2FByeonghyeon/filtered_bh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

초기 설정

In [ ]:
!pip install efficientnet_pytorch

from google.colab import drive
drive.mount('/content/drive')


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=5efff18c26815390aac9cfa523d81a068cc3a5c5a748b2e473a1a8b3f0c034f5
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 80 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 9.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-py3-none-any.whl size=76566 sha256=7fbcf2fb2d7b5b0d5448edfed98ee4821379f46eb7519dc0b936ec0d68ec05ae
  Stored in directory: /root/.cache/pip/wheels/01/e7/bb/347dc0e510803d7116a13d592b10cc68262da56a8eec4dd72f
  Created wheel for pynndescent: filename=pynndescent-0.5.4-py3-none-any.whl size=52372 sha256=dece4aeb2e8db3ba70caa5e079e8946fc23e86bc7657c5aba9bab4bd333ce70b
  Stored in directory: /root/.cache/pip/wheels/d0/5b/62/3401692ddad12324249c774c4b15ccb046946021e2b581c043
Successfully built umap-learn 

In [ ]:
from efficientnet_pytorch import EfficientNet
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# from keras.applications.vgg16 import VGG16 
# from keras.models import Model
# from keras.applications.vgg16 import preprocess_input 
import os
import logging
import torch
import pickle
from tqdm import tqdm
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
filename = 'filtered_10'
data_path = '/content/drive/MyDrive/Sketch_RNN/Dataset/Filtered_img/'
cluster_list_path = '/content/drive/MyDrive/Sketch_RNN/Dataset/Filtered_img/cluster_list.txt'
feat_path = "/content/drive/MyDrive/skku_Clustering/features_hw_" + filename + ".npy"
filename_path = "/content/drive/MyDrive/skku_Clustering/filenames_hw_" + filename + ".npy"
save_path = '/content/drive/MyDrive/skku_Clustering/Result/'
test_path = '/content/drive/MyDrive/skku_Clustering/test_set/'

In [ ]:
os.chdir(data_path)
data_list = []
with os.scandir(data_path) as files:
    for file in files:
        if not file.name.endswith('.txt'):
            data_list.append(file.name)

cluster_list = open(cluster_list_path , 'r').read().split('\n') 

#transform
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

def feature_extraction(image, model):
    img = tfms(image.convert("RGB")).unsqueeze(0)
    features = model.extract_features(img)
    return features



# Test set 생성

In [ ]:
cluster_list = open(cluster_list_path, 'r').read().split('\n')

s = 2000
e = 2050
for label in cluster_list:
  path = test_path + label + '/'
  print(path)
  if os.path.isdir(path) == False:
    os.makedirs(path)
  os.chdir(path)
  images = np.load('/content/drive/MyDrive/Sketch_RNN/Dataset/test/' + label + '.npy')
  for i in range (s, e): 
    image = Image.fromarray(images[i].reshape(28, 28))
    image.save('test_' + label + '_' + str(i - s + 1) + '.jpg')

/content/drive/MyDrive/skku_Clustering/test_set/cat/
/content/drive/MyDrive/skku_Clustering/test_set/circle/
/content/drive/MyDrive/skku_Clustering/test_set/cloud/
/content/drive/MyDrive/skku_Clustering/test_set/dog/
/content/drive/MyDrive/skku_Clustering/test_set/ear/
/content/drive/MyDrive/skku_Clustering/test_set/eye/
/content/drive/MyDrive/skku_Clustering/test_set/moustache/
/content/drive/MyDrive/skku_Clustering/test_set/mouth/
/content/drive/MyDrive/skku_Clustering/test_set/nose/
/content/drive/MyDrive/skku_Clustering/test_set/triangle/


# Clustering algorithm 설정

In [62]:
_eps = 1.3
_ms = 2

def clustering_method(cmd, cluster_num, input):
  if cmd == 'kmeans':
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=cluster_num, n_init = 20, n_jobs=-1, random_state = 0)
    kmeans.fit(input)
    return kmeans.labels_

  elif cmd == 'minibatch':
    from sklearn.cluster import MiniBatchKMeans
    mb_kmeans = MiniBatchKMeans(n_clusters=cluster_num,random_state=0, batch_size=6)
    mb_kmeans.fit(input)
    return mb_kmeans.labels_

  elif cmd == 'dbscan':
    from sklearn.cluster import DBSCAN
    dbscan = DBSCAN(eps=_eps, min_samples=_ms) # eps 1.8이면 63프로
    dbscan.fit(input)
    return dbscan.labels_
  
  elif cmd == 'optics':
    from sklearn.cluster import OPTICS, cluster_optics_dbscan
    optics = OPTICS(min_samples = 5)
    optics.fit(input)
    return optics.labels_

In [ ]:
from torch import nn
model = EfficientNet.from_pretrained('efficientnet-b0')
data = {}
noise = 0
correct = 0
wrong = 0
avg_pooling = nn.AdaptiveAvgPool2d(1)

os.chdir(test_path)
test_list = []
with os.scandir(test_path) as files:
  for file in files:
    if not file.name.startswith('.'):
      test_list.append(file.name)

for folder in test_list:
  test_images = os.listdir(test_path + folder)
  for i, image_name in enumerate(test_images):
    # train data의 feature vector load
    feat = np.load(feat_path)

    # test data에 대하여 feature 추출
    image = Image.open(test_path + folder +'/'+ image_name)
    new_feat = feature_extraction(image, model)
    new_feat = avg_pooling(new_feat)
    new_feat = new_feat.detach().numpy().reshape(-1)
    new_feat = new_feat.reshape(1, -1)

    # 기존 feature vector에 append. 
    feat = np.append(feat, new_feat, axis = 0)
    # print(feat.shape) # 608이 나와야 함

    # 차원 축소(t-SNE)
    feat = tsne.fit_transform(feat)

    # clustering
    labels = clustering_method('dbscan', len(cluster_list), feat)

    # 결과 보는 단계
    filenames = np.load(filename_path)
    filenames = np.append(filenames,image_name)

    groups = {}
    # holds the cluster id and the images { id: [images] }
    i = 0
    for f, cluster in zip(filenames, labels):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(f)
        else:
            groups[cluster].append(f)


    #make cluster_dict for calculate acc
    cluster_dict = {}
    for cluster in groups:
      image_count = []
      image_name = []
      for image in groups[cluster]:
        image_name.append(image.split('_')[0])
      for name in cluster_list:
        image_count.append(image_name.count(name))
      cluster_dict[cluster] = cluster_list[image_count.index(max(image_count))]
    
    #Acc
    from sklearn.metrics import f1_score
    pred = []
    gt = []    
    for cluster in groups:
        for food in groups[cluster]:
            pred.append(cluster_dict[cluster])
            gt.append(food.split('_')[0])
            if food.split('_')[0] == 'test':
              tmp = food.split('_')[1] + food.split('_')[2].split('.')[0]
              if cluster == -1:
                res = food.split('_')[1] + ' = ' + str(cluster)
                noise+=1
              else:
                res = food.split('_')[1] + ' = ' + cluster_dict[cluster]
                if food.split('_')[1] == cluster_dict[cluster]:
                  correct += 1
                else:
                  wrong += 1

    acc = str(round(f1_score(gt, pred,average='micro') * 100, 2))
    print(tmp + ' ->  res: ' + res + '    ACC: ' + acc + '    #cluster: ' + str(len(cluster_dict)))
    print('\t\t\t', end='')
    print(cluster_dict)

print('-----result-----')
print('total test: ', wrong + noise + correct)
print('noise: ', noise)
print('correct: ', correct)
print('wrong: ', wrong)



Loaded pretrained weights for efficientnet-b0
cat3 ->  res: cat = cat    ACC: 78.45    #cluster: 110
			{0: 'ear', 1: 'ear', 2: 'ear', 3: 'ear', 4: 'ear', 5: 'ear', -1: 'cat', 6: 'ear', 7: 'dog', 8: 'ear', 9: 'ear', 10: 'ear', 11: 'circle', 12: 'circle', 13: 'cat', 14: 'cat', 15: 'cat', 16: 'cat', 17: 'cat', 18: 'cat', 19: 'cat', 20: 'cat', 21: 'cat', 22: 'cat', 23: 'cat', 24: 'cat', 25: 'cat', 26: 'cat', 27: 'cat', 28: 'cat', 29: 'cat', 30: 'cat', 31: 'nose', 32: 'cat', 33: 'dog', 34: 'cat', 35: 'dog', 36: 'cat', 37: 'cat', 38: 'cat', 39: 'cat', 40: 'cat', 41: 'dog', 42: 'dog', 43: 'dog', 44: 'dog', 45: 'dog', 46: 'cloud', 47: 'mouth', 48: 'dog', 49: 'dog', 50: 'dog', 51: 'dog', 52: 'eye', 53: 'dog', 54: 'dog', 55: 'dog', 56: 'dog', 57: 'dog', 58: 'dog', 59: 'nose', 60: 'nose', 61: 'nose', 62: 'nose', 63: 'triangle', 64: 'nose', 65: 'nose', 66: 'nose', 67: 'nose', 68: 'triangle', 69: 'triangle', 70: 'cloud', 71: 'moustache', 72: 'moustache', 73: 'moustache', 74: 'moustache', 75: 'clou